## Import libraries

In [1]:
import tomopy
from helperFunctions import MoviePlotter
from tomoDataClass import tomoData
from helperFunctions import convert_to_numpy

## Create Simulated Data

In [2]:
#Set constants
numAngles = 200 #Number of projection angles
imageSize = 256 #Pixel size of one side of cubic array containing Shepp3D phantom

In [3]:
#Create a simulated object
obj = tomopy.shepp3d(size=imageSize)
print(obj.shape)
print("Simulated Object")
MoviePlotter(obj) #Plots vertical slices through the phantom

(256, 256, 256)
Simulated Object


Output()

In [4]:
# Set projection angles
angles = tomopy.angles(nang=numAngles, ang1=1, ang2=360)

# create projection images/sinogram
projections = tomopy.project(obj, angles, pad=False)

print("Projections through object")
MoviePlotter(projections)

Projections through object


Output()

In [ ]:
tomo = tomoData(projections)
tomo.jitter(maxShift = 7)
tomo.add_noise()
print("Projections through Object with jitter and noise")
tomo.makeNotebookProjMovie()

Projections through Object with jitter and noise


Output()

## Align Data

In [6]:
#Show bad reconstruction prior to alignment
tomo.reconstruct(algorithm='art')
print("\nBad reconstruction prior to alignment")
badRecon = tomo.recon.copy()
MoviePlotter(badRecon)



Using CPU-based reconstruction. Algorithm:  art
Reconstruction completed.

Bad reconstruction prior to alignment


Output()

In [7]:
tomo.reset_workingProjections(x_size=imageSize, y_size=imageSize) #You can adjust these for tighter cropping
tomo.normalize(isPhaseData=False)
tomo.cross_correlate_align(tolerance=0.1, max_iterations=15, stepRatio=1, yROI_Range=None, xROI_Range=None)
tomo.PMA(max_iterations=5, tolerance=0.05, algorithm='art', crop_bottom_center_y=imageSize, crop_bottom_center_x=imageSize, standardize=False)
tomo.make_updates_shift()



Normalizing projections


Cross-Correlation Alignment


Iteration 1/max_iterations: 100%|██████████| 200/200 [00:01<00:00, 156.15it/s]


Average pixel shift of iteration 1: 5.215427516927926


Iteration 2/max_iterations: 100%|██████████| 200/200 [00:01<00:00, 183.50it/s]


Average pixel shift of iteration 2: 0.012071067811865475


...

PMA Algorithm iterations:  60%|██████    | 3/5 [00:22<00:15,  7.64s/it]

Average pixel shift of iteration 2- total:0.06, just in x:0.06, just in y: 0.0


PMA Algorithm iterations:  60%|██████    | 3/5 [00:31<00:21, 10.58s/it]


Average pixel shift of iteration 3- total:0.015, just in x:0.015, just in y: 0.0
Convergence reached after 4 iterations.


Apply shifts to final projections: 100%|██████████| 200/200 [00:00<00:00, 300.60it/s]


In [8]:
tomo.makeNotebookProjMovie()

Output()

In [9]:
tomo.reconstruct(algorithm='art')
print("\nGood Reconstruction after alignment")
tomo.makeNotebookReconMovie()



Using CPU-based reconstruction. Algorithm:  art
Reconstruction completed.

Good Reconstruction after alignment


Output()